### Word Ngram Sentence Data Prepare And Select

In [38]:
#!pip install openpyxl

In [39]:
import os
import unicodedata
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil

In [40]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [41]:
# custom character alphabet for word
tr = re.compile(r"[abcçdefgğhıijklmnoöprsştuüvyzqxw]+", re.IGNORECASE|re.UNICODE) # Turkish filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n0123456789'
en = re.compile(r"[abcdefghıijklmnopqrstxuvwyz]+", re.IGNORECASE|re.UNICODE) # English
nl = re.compile(r"[abcdefghıijklmnopqrstxuvwyzāăēĕīĭōŏūŭ]+", re.IGNORECASE|re.UNICODE) # Dutch (Flemenk)
fr = re.compile(r"[abcçdefghıijklmnopqrstxuvwyzàâæèéêëîïôœùûüÿ]+", re.IGNORECASE|re.UNICODE) # French
de = re.compile(r"[abcdefghıijklmnopqrstxuvwyzäöüß]+", re.IGNORECASE|re.UNICODE) # German
es = re.compile(r"[abcdefghıijklmnopqrstxuvwyzñáéíóú]+", re.IGNORECASE|re.UNICODE) # Spanish (¿¡)
pt = re.compile(r"[abcçdefghıijklmnopqrstxuvwyzàáâãéêíóôõú]+", re.IGNORECASE|re.UNICODE) # Portuguese
it = re.compile(r"[abcdefghıijklmnopqrstxuvwyzàéèìòùî]+", re.IGNORECASE|re.UNICODE) # Italian
ar = re.compile(r"[ٿصؼۤڳڲؿڎػڠجڿ٬ٸؽؒؓطۄڀۂؘؔتٚڛےٝڜؖڦ٫ډ۰زۇٖۀ،لۓعٮێڔ۶ؚۧۜڤۏإٞٷؗۖ؈ژۣؕؑٴأۻڸۺگاڴڹۯ؉ْڌ؍ي؟ـٟړۅؐڶُىڽېًۢؠضۚڄٛڏٱۦ٩س٦ڼڂٔۘ٠ښٌٍ۬ٳ۾ٲږذۋٵٜ٘ڞڅںٗهڣۿپڒۥۗڋیؙم؞ثۨٹڵڪظٶۭ١ڭەڨحٕ؎ٺڷٰ۪۫ڻڥۛڑڟټآڡغګ؊ّٯڧڮ؏ۮ؋ؤ٪ؾڗۼق۟دکوِڰڐۃ۽ہفرڇچڝ۴بۈٽڕۡھةٓڃئ؛ڬٙڙڢڱۊَۆۉځ۠ۍۑۙڊنءڈٻشڍ؇۵كخ\ا]+", re.IGNORECASE|re.UNICODE) # Arabic

In [42]:
folder_name = "German"  # English, Turkish, German, French, Spanish, Arabic, Portuguese, Dutch, Italian
 
if folder_name == "English":
    custom_alp = en
elif folder_name == "Turkish":
    custom_alp = tr
elif folder_name == "German":
    custom_alp = de
elif folder_name == "French":
    custom_alp = fr
elif folder_name == "Spanish":
    custom_alp = es
elif folder_name == "Arabic":
    custom_alp = ar
elif folder_name == "Portuguese":
    custom_alp = pt
elif folder_name == "Dutch":
    custom_alp = nl
elif folder_name == "Italian":
    custom_alp = it
else:
    pass
#custom_alp = tr  # en, tr, de, fr, es, ar, pt, nl, it
print(f"{folder_name}\n{custom_alp}") 

German
re.compile('[abcdefghıijklmnopqrstxuvwyzäöüß]+', re.IGNORECASE)


In [43]:
def convert_one_character_letter(text):
    '''
    This function converts two byte occupy of letter to one byte unicode character without any visual change \n
    like as Turkish character ç,ş,ö,ğ. \n
    convert_one_character_letter(text): text is any string word or sentence.
    '''
    new_text = unicodedata.normalize('NFC', f"{text}")
    return new_text

In [44]:
def clean_text(text, custom_alp):  # def clean_text(text) for word
    '''
    clean_text(text, custom_alp) text is any string word or sentence.\n
    function clean string according to specific language alphabet\n
    '''
    #text_clean = re.findall(custom_alp, text)
    text_one = convert_one_character_letter(text) 
    text_clean = re.findall(custom_alp, str(text_one))
    text_result = "".join(text_clean)
    return text_result

In [62]:
def clean_text2(text, custom_alp):  # for ngram and sentence
    '''
    clean_text2(text, custom_alp) text is any string word or sentence.\n
    function clean string according to specific language alphabet\n
    '''
    #text_clean = re.findall(custom_alp, text)
    text_one = convert_one_character_letter(text) 
    text_clean = re.findall(custom_alp, str(text_one))
    text_result = " ".join(text_clean)  # difference from before
    return text_result

In [45]:
def string_length(string):
    len_out = len(string)
    return len_out

In [46]:
def lower_strip_func(x):
    try:
        var_low = x.lower()
        var_out = var_low.strip()
    except:
        var_out = x
    return var_out 

In [47]:
# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 1000  # 28 native word end index

# sentence check
twogram_sentence_check = True  # True, False
threegram_sentence_check = True
fourgram_sentence_check = True
fivegram_sentence_check = True
#sixgram_sentence_check = True
#sevengram_sentence_check = True
#eightgram_sentence_check = True
#ninegram_sentence_check = True
#tengram_sentence_check = True

# n gram select
twogram_select_start = 0
twogram_select_end = 200

threegram_select_start = 0
threegram_select_end = 200

fourgram_select_start = 0
fourgram_select_end = 200

fivegram_select_start = 0
fivegram_select_end = 200

#sixgram_select_start = 0
#sixgram_select_end = 200

#sevengram_select_start = 0
#sevengram_select_end = 200

#eightgram_select_start = 0
#eightgram_select_end = 200

#ninegram_select_start = 0
#ninegram_select_end = 200

#tengram_select_start = 0
#tengram_select_end = 200

sentence_select_start = 0
sentence_select_end = 400

In [48]:
df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.capitalize()}/Result/Word/Merge/Word_Merge2.xlsx")
#df_word = pd.read_csv("")
df_word.drop_duplicates(inplace=True)
df_word

,word,frequency
0,ich,8138198
1,sie,5307295
2,das,4314201
3,ist,4200394
4,du,4093352
...,...,...
274529,galapremiere,5
274530,gemurkse,5
274531,granet,5
274532,bürgernahen,5


In [49]:
df_word.iloc[:,0] = df_word.iloc[:,0].apply(lambda x: clean_text(x, custom_alp))
df_word.dropna(inplace=True)
df_word.drop(df_word[df_word.iloc[:,0] == ""].index, inplace=True)
#df_word.drop(df_word[df_word.iloc[:,0] == "fikret"].index, inplace=True) # paris, fikret
df_word["word"] = df_word["word"].apply(lambda x: lower_strip_func(x))
df_word.drop_duplicates(subset="word", inplace=True)
df_word.reset_index(drop=True, inplace=True)
df_word

,word,frequency
0,ich,8138198
1,sie,5307295
2,das,4314201
3,ist,4200394
4,du,4093352
...,...,...
274527,galapremiere,5
274528,gemurkse,5
274529,granet,5
274530,bürgernahen,5


In [50]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [51]:
df_word = df_word[~df_word.iloc[:,0].isin(disable_list)]
df_word

,word,frequency
0,ich,8138198
1,sie,5307295
2,das,4314201
3,ist,4200394
4,du,4093352
...,...,...
274527,galapremiere,5
274528,gemurkse,5
274529,granet,5
274530,bürgernahen,5


In [52]:
df_word_select = df_word.iloc[word_start:word_end,]
df_word_select

,word,frequency
0,ich,8138198
1,sie,5307295
2,das,4314201
3,ist,4200394
4,du,4093352
...,...,...
996,boot,18087
997,findest,18085
998,fahr,17973
999,anfang,17971


In [53]:
df_word_select.to_excel(f"/home/kurubal/Downloads/{folder_name.capitalize()}_Word_Selected_{word_end}.xlsx", index=False)

##### Ngrams Select

In [63]:
if twogram_sentence_check:
    df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_twogram = df_twogram_sent.loc[:,["twogram","frequency"]]
    #df_twogram_select = df_twogram.iloc[twogram_start:twogram_end,]
else:
    df_twogram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.capitalize()}/Result/N Gram/Merge/Twogram_Merge.csv")  
    df_twogram = df_twogram.loc[:,["twogram","frequency"]]
    #df_twogram_select = df_twogram.iloc[twogram_start:twogram_end,]

df_twogram = df_twogram[df_twogram["frequency"] > 5]
df_twogram

,twogram,frequency
0,ich weiß,38274
1,komm schon,34485
2,alles klar,24441
3,oh gott,23763
4,in ordnung,23303
...,...,...
61368,bob ford,6
61369,dame spielen,6
61370,bruder recep,6
61371,hallo clarissa,6


In [64]:
df_twogram.iloc[:,0] = df_twogram.iloc[:,0].apply(lambda x: clean_text2(x, custom_alp))
df_twogram.dropna(inplace=True)
df_twogram.drop(df_twogram[df_twogram.iloc[:,0] == ""].index, inplace=True)
df_twogram["twogram"] = df_twogram["twogram"].apply(lambda x: lower_strip_func(x))
df_twogram.drop_duplicates(subset="twogram", inplace=True)
df_twogram.reset_index(drop=True, inplace=True)
df_twogram

,twogram,frequency
0,ich weiß,38274
1,komm schon,34485
2,alles klar,24441
3,oh gott,23763
4,in ordnung,23303
...,...,...
60859,bob ford,6
60860,dame spielen,6
60861,bruder recep,6
60862,hallo clarissa,6


In [65]:
df_twogram_select = df_twogram.iloc[twogram_select_start:twogram_select_end,]
df_twogram_select

,twogram,frequency
0,ich weiß,38274
1,komm schon,34485
2,alles klar,24441
3,oh gott,23763
4,in ordnung,23303
...,...,...
195,sehr erfreut,2123
196,verpiss dich,2120
197,na dann,2119
198,ich bin,2119


In [23]:
df_twogram_select.to_excel(f"/home/kurubal/Downloads/{folder_name.capitalize()}_Twogram_Selected_{twogram_select_end}.xlsx", index=False)

In [66]:
if threegram_sentence_check:
    df_threegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.capitalize()}/Result/N Gram/N Gram And Sentence/Three_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_threegram_sent.rename(columns={"three_gram":"threegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_threegram = df_threegram_sent.loc[:,["threegram","frequency"]]
else:
    df_threegram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.capitalize()}/Result/N Gram/Merge/Threegram_Merge.csv")  
    df_threegram = df_threegram.loc[:,["threegram","frequency"]]

df_threegram

,threegram,frequency
0,tut mir leid,38503
1,was ist los,32284
2,was ist das,31321
3,oh mein gott,29191
4,ich weiß nicht,29160
...,...,...
443475,immer nur geschichten,2
443476,dies ist phoebe,2
443477,eine bemerkenswert große,2
443478,medizinische oder chirurgie,2


In [68]:
df_threegram.iloc[:,0] = df_threegram.iloc[:,0].apply(lambda x: clean_text2(x, custom_alp))
df_threegram.dropna(inplace=True)
df_threegram.drop(df_threegram[df_threegram.iloc[:,0] == ""].index, inplace=True)
df_threegram["threegram"] = df_threegram["threegram"].apply(lambda x: lower_strip_func(x))
df_threegram.drop_duplicates(subset="threegram", inplace=True)
df_threegram.reset_index(drop=True, inplace=True)
df_threegram

,threegram,frequency
0,tut mir leid,38503
1,was ist los,32284
2,was ist das,31321
3,oh mein gott,29191
4,ich weiß nicht,29160
...,...,...
440293,immer nur geschichten,2
440294,dies ist phoebe,2
440295,eine bemerkenswert große,2
440296,medizinische oder chirurgie,2


In [69]:
df_threegram_select = df_threegram.iloc[threegram_select_start:threegram_select_end,]
df_threegram_select

,threegram,frequency
0,tut mir leid,38503
1,was ist los,32284
2,was ist das,31321
3,oh mein gott,29191
4,ich weiß nicht,29160
...,...,...
195,das ist unglaublich,1435
196,was bisher geschah,1430
197,ich erinnere mich,1420
198,das ist großartig,1419


In [33]:
df_threegram_select.to_excel(f"/home/kurubal/Downloads/{folder_name.capitalize()}_Threegram_Selected_{threegram_select_end}.xlsx", index=False)

In [71]:
if fourgram_sentence_check:
    df_fourgram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.capitalize()}/Result/N Gram/N Gram And Sentence/Four_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_fourgram_sent.rename(columns={"four_gram":"fourgram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_fourgram = df_fourgram_sent.loc[:,["fourgram","frequency"]]
else:
    df_fourgram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.capitalize()}/Result/N Gram/Merge/Fourgram_Merge.csv")  
    df_fourgram = df_fourgram.loc[:,["fourgram","frequency"]]

df_fourgram

,fourgram,frequency
0,es tut mir leid,24980
1,ich weiß es nicht,19408
2,was machst du da,6686
3,was machst du hier,6298
4,was soll das heißen,5727
...,...,...
744768,klarinette fagott und oboe,2
744769,ich sage sie an,2
744770,gib s her winzling,2
744771,von mir aus hier,2


In [72]:
df_fourgram.iloc[:,0] = df_fourgram.iloc[:,0].apply(lambda x: clean_text2(x, custom_alp))
df_fourgram.dropna(inplace=True)
df_fourgram.drop(df_fourgram[df_fourgram.iloc[:,0] == ""].index, inplace=True)
df_fourgram["fourgram"] = df_fourgram["fourgram"].apply(lambda x: lower_strip_func(x))
df_fourgram.drop_duplicates(subset="fourgram", inplace=True)
df_fourgram.reset_index(drop=True, inplace=True)
df_fourgram

,fourgram,frequency
0,es tut mir leid,24980
1,ich weiß es nicht,19408
2,was machst du da,6686
3,was machst du hier,6298
4,was soll das heißen,5727
...,...,...
742586,klarinette fagott und oboe,2
742587,ich sage sie an,2
742588,gib s her winzling,2
742589,von mir aus hier,2


In [76]:
df_fourgram_select = df_fourgram.iloc[fourgram_select_start:fourgram_select_end,]
df_fourgram_select

,fourgram,frequency
0,es tut mir leid,24980
1,ich weiß es nicht,19408
2,was machst du da,6686
3,was machst du hier,6298
4,was soll das heißen,5727
...,...,...
195,gib mir deine hand,835
196,wo gehen wir hin,834
197,warum tust du das,833
198,und weißt du was,829


In [44]:
df_fourgram_select.to_excel(f"/home/kurubal/Downloads/{folder_name.capitalize()}_Fourgram_Selected_{fourgram_select_end}.xlsx", index=False)

In [73]:
if fivegram_sentence_check:
    df_fivegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.capitalize()}/Result/N Gram/N Gram And Sentence/Five_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_fivegram_sent.rename(columns={"five_gram":"fivegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_fivegram = df_fivegram_sent.loc[:,["fivegram","frequency"]]
else:
    df_fivegram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.capitalize()}/Result/N Gram/Merge/Fivegram_Merge.csv")  
    df_fivegram = df_fivegram.loc[:,["fivegram","frequency"]]

df_fivegram

,fivegram,frequency
0,es tut mir so leid,5224
1,was ist los mit dir,2971
2,was machst du denn hier,2582
3,ich bin gleich wieder da,2315
4,was zum teufel ist das,1841
...,...,...
793043,carole ist wie ein paris,2
793044,caribbean banking unlimited niederländische paris,2
793045,carola da ist n paris,2
793046,bist du ein monroe paris,2


In [74]:
df_fivegram.iloc[:,0] = df_fivegram.iloc[:,0].apply(lambda x: clean_text2(x, custom_alp))
df_fivegram.dropna(inplace=True)
df_fivegram.drop(df_fivegram[df_fivegram.iloc[:,0] == ""].index, inplace=True)
df_fivegram["fivegram"] = df_fivegram["fivegram"].apply(lambda x: lower_strip_func(x))
df_fivegram.drop_duplicates(subset="fivegram", inplace=True)
df_fivegram.reset_index(drop=True, inplace=True)
df_fivegram

,fivegram,frequency
0,es tut mir so leid,5224
1,was ist los mit dir,2971
2,was machst du denn hier,2582
3,ich bin gleich wieder da,2315
4,was zum teufel ist das,1841
...,...,...
791749,carole ist wie ein paris,2
791750,caribbean banking unlimited niederländische paris,2
791751,carola da ist n paris,2
791752,bist du ein monroe paris,2


In [75]:
df_fivegram_select = df_fivegram.iloc[fivegram_select_start:fivegram_select_end,]
df_fivegram_select

,fivegram,frequency
0,es tut mir so leid,5224
1,was ist los mit dir,2971
2,was machst du denn hier,2582
3,ich bin gleich wieder da,2315
4,was zum teufel ist das,1841
...,...,...
195,was ist denn jetzt los,302
196,es war nicht meine schuld,301
197,das habe ich mir gedacht,300
198,ich habe auf dich gewartet,300


In [ ]:
df_fivegram_select.to_excel(f"/home/kurubal/Downloads/{folder_name.capitalize()}_Fivegram_Selected_{fivegram_select_end}.xlsx", index=False)

##### Sentence Select

In [77]:

df_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.capitalize()}/Result/Sentence/Merge/Sentence_Merge.csv")  
df_sentence = df_sentence.loc[:,["sentence","frequency"]]

df_sentence

,sentence,frequency
0,ja,486696
1,nein,345601
2,was,170799
3,danke,148807
4,hallo,95313
...,...,...
4076034,wurst wurst wurst,2
4076035,wir können relaxen uns vielleicht einen film a...,2
4076036,sie können fliegen haben den röntgenblick,2
4076037,was tust du cookie,2


In [78]:
df_sentence.iloc[:,0] = df_sentence.iloc[:,0].apply(lambda x: clean_text2(x, custom_alp))
df_sentence.dropna(inplace=True)
df_sentence.drop(df_sentence[df_sentence.iloc[:,0] == ""].index, inplace=True)
df_sentence["sentence"] = df_sentence["sentence"].apply(lambda x: lower_strip_func(x))
df_sentence.drop_duplicates(subset="sentence", inplace=True)
df_sentence.reset_index(drop=True, inplace=True)
df_sentence

,sentence,frequency
0,ja,486696
1,nein,345601
2,was,170799
3,danke,148807
4,hallo,95313
...,...,...
4050288,wurst wurst wurst,2
4050289,wir können relaxen uns vielleicht einen film a...,2
4050290,sie können fliegen haben den röntgenblick,2
4050291,was tust du cookie,2


In [79]:
df_sentence_select = df_sentence.iloc[sentence_select_start:sentence_select_end,]
df_sentence_select

,sentence,frequency
0,ja,486696
1,nein,345601
2,was,170799
3,danke,148807
4,hallo,95313
...,...,...
295,ich muss gehen,3947
296,danke sehr,3912
297,ich verstehe nicht,3888
298,unmöglich,3851


In [ ]:
df_sentence_select.to_excel(f"/home/kurubal/Downloads/{folder_name.capitalize()}_Sentence_Selected_{sentence_select_end}.xlsx", index=False)